In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

def extract_data(start, count):
    url_da_api = f"https://store.steampowered.com/search/results/?query&start={start}&count={count}&dynamic_data=&sort_by=_ASC&supportedlang=brazilian%2Cenglish&category1=998&os=linux%2Cmac%2Cwin&snr=1_7_7_230_7&infinite=1"

    response = requests.get(url_da_api)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        links = soup.find_all("a")

        urls = []
        ids = []
        nomes = []

        for link in links:
            url = link.get("href").replace('"', '')

            id_jogo = url.split("/")[4]

            nome_jogo = url.split("/")[-2].replace("_", " ")

            urls.append(url)
            ids.append(id_jogo)
            nomes.append(nome_jogo)

        df = pd.DataFrame({"URL": urls, "ID do Jogo": ids, "Nome do Jogo": nomes})
        df = df.applymap(lambda x: x.replace("\\", "") if isinstance(x, str) else x)

        return df

    else:
        print(f"A solicitação falhou com o status code {response.status_code}")
        return None

start = 0
count = 100

result_df = pd.DataFrame()

while start <= 100:
    df_chunk = extract_data(start, count)

    if df_chunk is not None:
        result_df = pd.concat([result_df, df_chunk], ignore_index=True)
        start += count
    else:
        break

print(result_df)


In [ ]:
def get_app_details(appid):
    url = f"http://store.steampowered.com/api/appdetails/?appids={appid}"
    response = requests.get(url)
    data = response.json()

    if data and str(appid) in data:
        app_data = data[str(appid)]
        if app_data and 'data' in app_data:
            return app_data['data']

    return None

column_mapping = {
    'type': 'Tipo',
    'required_age': 'Idade Requerida',
    'is_free': 'É Grátis?',
    'detailed_description': 'Descrição Detalhada',
    'about_the_game': 'Sobre o Jogo',
    'short_description': 'Descrição Curta',
    'supported_languages': 'Línguas',
    'header_image': 'Imagem Header',
    'capsule_image': 'Imagem Capsule',
    'capsule_imagev5': 'Imagem Capsule V5',
    'website': 'Site',
    'ext_user_account_notice': 'Aviso User Externo',
    'developers': 'Desenvolvedores',
    'publishers': 'Publicadores',
    'platforms': 'Plataformas',
    'categories': 'Categorias',
    'genres': 'Gêneros',
    'release_date': 'Data de lançamento'
}

desired_objects = list(column_mapping.keys())

for obj in desired_objects:
    result_df[column_mapping[obj]] = None

for index, row in result_df.iterrows():
    appid = row['ID do Jogo']
    app_details = get_app_details(appid)

    if app_details:
        for obj in desired_objects:
            if obj in app_details and obj not in ['name', 'steam_appid']:
                result_df.at[index, column_mapping[obj]] = app_details[obj]

print(result_df.head())


In [ ]:
def get_api_info(appid):
    url = f"https://store.steampowered.com/appreviews/{appid}?json=1"
    try:
        response = requests.get(url)
        data = response.json()

        if 'success' in data and data['success'] == 1:
            query_summary = data['query_summary']
            total_reviews = query_summary.get('total_reviews', 0)
            total_negative = query_summary.get('total_negative', 0)
            total_positive = query_summary.get('total_positive', 0)
            review_score_desc = query_summary.get('review_score_desc', '')

            return total_reviews, total_negative, total_positive, review_score_desc
        else:
            return None, None, None, None

    except Exception as e:
        return None, None, None, f"Error: {str(e)}"


api_column_mapping = {
    'total_reviews': 'Total de Reviews',
    'total_negative': 'Total de Reviews Negativas',
    'total_positive': 'Total de Reviews Positivas',
    'review_score_desc': 'Descrição de Reviews'
}

for col in api_column_mapping.values():
    result_df[col] = None

result_df['Error'] = ""

for index, row in result_df.iterrows():
    appid = row['ID do Jogo']
    total_reviews, total_negative, total_positive, review_score_desc = get_api_info(appid)

    if total_reviews is not None:
        for key, col_name in api_column_mapping.items():
            result_df.at[index, col_name] = locals()[key]
    else:
        result_df.at[index, 'Error'] = "Chamada à API falhou"

print(result_df)



In [ ]:
def extrair_texto_html(html):
    if pd.notna(html):
        soup = BeautifulSoup(html, 'html.parser')
        return soup.get_text()
    else:
        return html

colunas_para_tratar = ['Descrição Detalhada', 'Sobre o Jogo', 'Línguas']

for coluna in colunas_para_tratar:
    result_df[coluna] = result_df[coluna].apply(extrair_texto_html)

print(result_df)


In [ ]:
def tratar_plataformas(plataformas_dict):
    if pd.notna(plataformas_dict):
        plataformas = [plataforma.capitalize() for plataforma, disponivel in plataformas_dict.items() if disponivel]
        return ', '.join(plataformas)
    else:
        return ''

result_df['Plataformas'] = result_df['Plataformas'].apply(tratar_plataformas)

print(result_df)


In [ ]:
def extrair_categorias(categorias_list):
    if categorias_list is None:
        return pd.Series({'Categoria Id': None, 'Categoria Descrição': None})

    ids = [categoria['id'] if categoria is not None else None for categoria in categorias_list]
    descricoes = [categoria['description'] if categoria is not None else None for categoria in categorias_list]
    return pd.Series({'Categoria Id': ', '.join(map(str, ids)), 'Categoria Descrição': ', '.join(descricoes)})

def extrair_generos(generos_list):
    if generos_list is None:
        return pd.Series({'Gênero Id': None, 'Gênero Descrição': None})

    ids = [genero['id'] if genero is not None else None for genero in generos_list]
    descricoes = [genero['description'] if genero is not None else None for genero in generos_list]
    return pd.Series({'Gênero Id': ', '.join(map(str, ids)), 'Gênero Descrição': ', '.join(descricoes)})

result_df[['Categoria Id', 'Categoria Descrição']] = result_df['Categorias'].apply(extrair_categorias)
result_df[['Gênero Id', 'Gênero Descrição']] = result_df['Gêneros'].apply(extrair_generos)

print(result_df)


In [ ]:
def extrair_data_lancamento(data_dict):
    if pd.isna(data_dict):
        return pd.Series({'Lançará em Breve': None, 'Data de Lançamento': None})

    return pd.Series({'Lançará em Breve': data_dict.get('coming_soon'), 'Data de Lançamento': data_dict.get('date')})

result_df[['Lançará em Breve', 'Data de Lançamento']] = result_df['Data de lançamento'].apply(extrair_data_lancamento)

result_df['Data de Lançamento'] = pd.to_datetime(result_df['Data de Lançamento'], errors='coerce')

result_df['Data de Lançamento'] = result_df['Data de Lançamento'].dt.strftime('%d/%m/%Y')

print(result_df)


In [ ]:
def transformar_em_string(valor):
    if isinstance(valor, list):
        return ', '.join(valor)
    else:
        return valor

result_df['Desenvolvedores'] = result_df['Desenvolvedores'].apply(transformar_em_string)
result_df['Publicadores'] = result_df['Publicadores'].apply(transformar_em_string)

print(result_df)

In [26]:
result_df = result_df.drop(['Categorias', 'Gêneros', 'Data de lançamento', 'Error'], axis=1)

result_df.to_csv('dataset.csv')